In [1]:
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, LSTM, LeakyReLU, Dropout
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import tensorflow as tf


data = pd.read_csv("./Output/simulation_Data_2020.csv")
data.head()

In [1]:
data = pd.read_csv("./Output/simulation_Data_2020.csv")
data.head()

NameError: name 'pd' is not defined

In [3]:
power_cooling = data[['power_cooling']]
power_heating_data = go.Scatter(x = data['Date_Time'], y = data['power_cooling'])
layout = go.Layout(title ='power_cooling time series', xaxis = dict(title='Date'), yaxis = dict(title='power_cooling'))

fig = go.Figure(data = [power_heating_data], layout = layout)
fig.write_html('2Scatter_power_cooling.html', auto_open = True)

In [4]:
min_max_scaler = MinMaxScaler()
norm_data = min_max_scaler.fit_transform(power_cooling.values)
norm_data
norm_data_df = pd.DataFrame(norm_data)
norm_data_df.head()

,0
0,0.114461
1,0.367393
2,0.412619
3,0.439978
4,0.433277


In [5]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i+target_size])
    return np.array(data), np.array(labels)

In [6]:
past_history = 5
future_target = 0

TRAIN_SPLIT = int(len(norm_data) * 0.8)


x_train, y_train = univariate_data(norm_data,
                                   0,
                                   TRAIN_SPLIT,
                                   past_history,
                                   future_target)

x_test, y_test = univariate_data(norm_data,
                                 TRAIN_SPLIT,
                                 None,
                                 past_history,
                                 future_target)

In [7]:
num_units = 64
learning_rate = 0.0001
activation_function = 'sigmoid'
adam = Adam(lr=learning_rate)
loss_function = 'mse'
batch_size = 5
num_epochs = 50

# Initialize the RNN
model = Sequential()
model.add(LSTM(units = num_units, activation=activation_function, input_shape=(None, 1)))
model.add(LeakyReLU(alpha=0.5))
model.add(Dropout(0.1))
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer=adam, loss=loss_function)

In [8]:
history = model.fit(
    x_train,
    y_train,
    validation_split=0.1,
    batch_size=batch_size,
    epochs=num_epochs,
    shuffle=False
)

Epoch 1/50
270/270 [==============================] - 6s 12ms/step - loss: 0.3509 - val_loss: 0.0175
Epoch 2/50
270/270 [==============================] - 2s 7ms/step - loss: 0.0428 - val_loss: 0.0171
Epoch 3/50
270/270 [==============================] - 1s 5ms/step - loss: 0.0358 - val_loss: 0.0169
Epoch 4/50
270/270 [==============================] - 1s 5ms/step - loss: 0.0369 - val_loss: 0.0167
Epoch 5/50
270/270 [==============================] - 2s 6ms/step - loss: 0.0354 - val_loss: 0.0168
Epoch 6/50
270/270 [==============================] - 1s 5ms/step - loss: 0.0355 - val_loss: 0.0162
Epoch 7/50
270/270 [==============================] - 1s 5ms/step - loss: 0.0303 - val_loss: 0.0155
Epoch 8/50
270/270 [==============================] - 1s 5ms/step - loss: 0.0318 - val_loss: 0.0161
Epoch 9/50
270/270 [==============================] - 1s 5ms/step - loss: 0.0314 - val_loss: 0.0158
Epoch 10/50
270/270 [==============================] - 2s 6ms/step - loss: 0.0292 - val_loss: 0.015

In [9]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))
epochs_list = [*range(0, 50, 1)]

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = epochs_list, y = loss, mode = 'lines', name = 'Loss'))
fig2.add_trace(go.Scatter(x = epochs_list, y = val_loss, mode = 'lines', name = 'Loss Val'))
fig2.update_layout(
    title="Power cooling Loss VS Loss Val",
    xaxis_title="Epochs",
    yaxis_title="Loss and Loss Val",
#     legend_title="Legend Title",
)
# layout2 = go.Layout(title ='Power heating Loss VS Loss Val')
fig2.write_html('Power cooling Loss VS Los Val.html', auto_open = True)

In [10]:
original = pd.DataFrame(min_max_scaler.inverse_transform(y_test))
predictions = pd.DataFrame(min_max_scaler.inverse_transform(model.predict(x_test)))

In [11]:
predictions

,0
0,0.147307
1,0.147307
2,0.147307
3,0.147307
4,0.147307
...,...
366,0.147307
367,0.147307
368,0.298367
369,0.452473


In [12]:
original

,0
0,0.00
1,0.00
2,0.00
3,0.00
4,0.00
...,...
366,0.00
367,0.41
368,0.73
369,3.84


In [13]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x = original.index, y = original[0], mode = 'lines', name = 'Test Data'))
fig3.add_trace(go.Scatter(x = predictions.index, y = predictions[0], mode = 'lines', name = 'Predictions'))
fig3.update_layout(
    title="Original VS Predictions",
    xaxis_title="Days",
    yaxis_title="Original and Predictions",
#     legend_title="Legend Title",
)
# layout2 = go.Layout(title ='Power heating Loss VS Loss Val')
fig3.write_html('Original VS Predictions Power cooling.html', auto_open = True)